# Titanic #2


In [148]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [149]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [150]:
PATH = "data/titanic/"

In [151]:
df_raw = pd.read_csv(f'{PATH}train.csv')

In [152]:
df_raw.isnull().sum() / len(df_raw.isnull())

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.198653
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.771044
Embarked       0.002245
dtype: float64

In [153]:
df_raw.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [154]:
df_raw.drop('Cabin', axis=1, inplace=True)

In [155]:
df_raw.isnull().sum() / len(df_raw.isnull())

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.198653
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Embarked       0.002245
dtype: float64

In [156]:
df_raw.Age.fillna(df_raw['Age'].mean(), inplace=True)

In [157]:
df_raw.dropna(inplace=True)

In [158]:
df_raw.isnull().sum() / len(df_raw.isnull())

PassengerId    0.0
Survived       0.0
Pclass         0.0
Name           0.0
Sex            0.0
Age            0.0
SibSp          0.0
Parch          0.0
Ticket         0.0
Fare           0.0
Embarked       0.0
dtype: float64

In [159]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Name         889 non-null    object 
 4   Sex          889 non-null    object 
 5   Age          889 non-null    float64
 6   SibSp        889 non-null    int64  
 7   Parch        889 non-null    int64  
 8   Ticket       889 non-null    object 
 9   Fare         889 non-null    float64
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 83.3+ KB


In [160]:
df_raw.drop(['PassengerId', 'Fare', 'Ticket', 'Name'], axis=1, inplace=True)

In [161]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Sex       889 non-null    object 
 3   Age       889 non-null    float64
 4   SibSp     889 non-null    int64  
 5   Parch     889 non-null    int64  
 6   Embarked  889 non-null    object 
dtypes: float64(1), int64(4), object(2)
memory usage: 55.6+ KB


In [162]:
df = pd.get_dummies(df_raw, columns=['Sex', 'Embarked'], drop_first=True)

In [163]:
df.head()

,Survived,Pclass,Age,SibSp,Parch,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,1,0,1
1,1,1,38.0,1,0,0,0,0
2,1,3,26.0,0,0,0,0,1
3,1,1,35.0,1,0,0,0,1
4,0,3,35.0,0,0,1,0,1


In [164]:
X = df.drop('Survived', axis=1)
y = df['Survived']

In [165]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.44, random_state=42)

In [166]:
m = RandomForestClassifier(n_jobs=-1)
%time m.fit(X_train, y_train)
m.score(X_test, y_test)

CPU times: user 143 ms, sys: 56.5 ms, total: 200 ms
Wall time: 140 ms


0.7678571428571429

In [167]:
p = {"n_estimators": np.arange(50, 1000, 50),
         "max_depth": np.arange(1, 100),
         "min_samples_split": np.arange(1, 50),
         "min_samples_leaf": np.arange(1, 10)}

tuned_model = RandomizedSearchCV(m, p, cv=5, n_jobs=-1)

%time tuned_model.fit(X_train, y_train)
tuned_model.score(X_test, y_test)

CPU times: user 1.25 s, sys: 434 ms, total: 1.68 s
Wall time: 11.1 s


0.8163265306122449

In [177]:
df_test = pd.read_csv(f'{PATH}test.csv')
df_test_raw = df_test

In [169]:
df_test.isnull().sum() / len(df_test)

PassengerId    0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.205742
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.002392
Cabin          0.782297
Embarked       0.000000
dtype: float64

In [170]:
df_test.Age.fillna(df_test['Age'].mean(), inplace=True)

In [171]:
df_test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [172]:
df_test.drop(['PassengerId', 'Fare', 'Ticket', 'Name', 'Cabin'], axis=1, inplace=True)


In [173]:
df_test.dropna(inplace=True)

In [174]:
df_test.isnull().sum() / len(df_test)

Pclass      0.0
Sex         0.0
Age         0.0
SibSp       0.0
Parch       0.0
Embarked    0.0
dtype: float64

In [175]:
df_test = pd.get_dummies(df_test, columns=['Sex', 'Embarked'], drop_first=True)

In [176]:
y_test = m.predict(df_test)

In [179]:
d = {'PassengerId': df_test_raw.PassengerId, 'Survived': y_test}
pd.DataFrame(d).to_csv(f'{PATH}res2.csv', index=False)